In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# load train and test datasets
train_df=pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test_df=pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
print('train data shape',train_df.shape)
train_df.head()

In [ ]:
print('test data shape',test_df.shape)
test_df.head()

### description of features
A SHORT DESCRIPTION OF THE FEATURES.

datetime - hourly date + timestamp

season - 1 = spring, 2 = summer, 3 = fall, 4 = winter

holiday - whether the day is considered a holiday

workingday - whether the day is neither a weekend nor holiday

weather -

1: Clear, Few clouds, Partly cloudy, Partly cloudy

2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist

3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

temp - temperature in Celsius

atemp - "feels like" temperature in Celsius

humidity - relative humidity

windspeed - wind speed

casual - number of non-registered user rentals initiated

registered - number of registered user rentals initiated

count - number of total rentals


## train and test data has no missing data we we are goog to go and do our edas and feature engineering 

In [ ]:
# checking misssing values
print('missing values in train data',train_df.isna().sum().sum())
print('missing values in test data', test_df.isna().sum().sum())

In [ ]:
# checking data types of train and test

train_df.dtypes

In [ ]:
test_df.dtypes

In [ ]:
# extract data rom date column (hour ,day of week , week of year )

train_df['datetime'] = pd.to_datetime(train_df['datetime'])

train_df['month'] = train_df["datetime"].dt.month
train_df["hour"] = train_df["datetime"].dt.hour
train_df["dayofweek"] = train_df["datetime"].dt.dayofweek

train_df.drop('datetime', axis=1, inplace=True)

train_df.head()




In [ ]:
# tranform test datetime



test_df['datetime'] = pd.to_datetime(test_df['datetime'])

test_df['month'] = test_df["datetime"].dt.month
test_df["hour"] = test_df["datetime"].dt.hour
test_df["dayofweek"] = test_df["datetime"].dt.dayofweek

test_df.drop('datetime', axis=1, inplace=True)
test_df.head()


# we have category features present as numeric 
we have to chage their dtypes 

In [ ]:
categoricalFeatureNames = ["season","holiday","workingday","weather","dayofweek","month","hour"]

for var in categoricalFeatureNames:
    train_df[var] = train_df[var].astype("category")
    test_df[var] = test_df[var].astype("category")

train_df.dtypes

In [ ]:
# dropping causaul and registered colums 
train_df.drop(['registered','casual'], axis=1, inplace=True)


In [ ]:
# now doing some eda

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# viewing correlation between features 
plt.figure(figsize=(10,10))
sns.heatmap(train_df.corr("pearson"),
            vmin=-1, vmax=1,
            cmap='coolwarm',
            annot=True, 
            square=True);

In [ ]:
# we see that temp and atemp are highly correlated we can drop either
train_df.drop('atemp', axis=1, inplace=True)
test_df.drop('atemp', axis=1, inplace=True)


train_df.head()

In [ ]:
# viewing count with temperature in every hour 
sns.lmplot(x="temp",hue='season',col='hour',y="count", data=train_df)

In [ ]:
# we cand see count varies with our as expected people take bikes to go work and go back

In [ ]:
train_df.groupby('hour').sum()['count'].plot.bar()
train_df.groupby('hour').sum()['count'].plot()

In [ ]:
# we can see that  rent is lowest in winter
train_df.groupby('season').sum()['count'].plot.pie()

In [ ]:

# itis clear that the count is very low in holiday
train_df.holiday.value_counts()
sns.factorplot(x='holiday',data=train_df,kind='count',size=5,aspect=1)
# majority of data is for non holiday days.



In [ ]:
# definig lmse function


def rmse_log(y_test,preds):
    return  (np.sqrt(np.mean(np.square(np.log(preds+1) - np.log(y_test+1)))))

In [ ]:
# converting count to log scale
train_df["count"] = np.log1p(train_df["count"])


In [ ]:
# splitting train df in train and validation
from sklearn.model_selection import train_test_split

y= train_df['count']
train_df.drop('count', axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(train_df ,y , test_size=0.27,random_state=42)

In [ ]:
from  sklearn.ensemble import  GradientBoostingRegressor
params = {'n_estimators': 6000, 'max_depth':4 , 'loss':'huber','max_features':'sqrt',
          'learning_rate' :0.01 ,'min_samples_leaf':15,
                                'min_samples_split':10}
gbm_model = GradientBoostingRegressor(**params)
gbm_model.fit(X_train,y_train)

x_train_pred = gbm_model.predict(X_train)
model_pred = gbm_model.predict(X_test)
submit = gbm_model.predict(test_df)
print("train result         ",rmse_log(np.expm1(y_train),np.expm1(x_train_pred)))
print("validation result    ",rmse_log(np.expm1(y_test),np.expm1(model_pred)))


In [ ]:
# transforming the prediction 
count_submit=np.round(np.expm1(submit))
df=pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')


In [ ]:
submission = pd.DataFrame({'datetime': df['datetime'],
                       'count': count_submit})
submission.to_csv('submission.csv', index=False)